<a href="https://colab.research.google.com/github/MKhromina/-furry-train/blob/master/chatbot_1.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install nltk

In [ ]:
# импорт библиотек для проекта
import pandas as pd
import nltk
nltk.download('punkt') 
nltk.download('wordnet')
nltk.download('averaged_perceptron_tagger')
nltk.download('stopwords')
import numpy as np
import re
from nltk.stem import wordnet 
from sklearn.feature_extraction.text import CountVectorizer 
from sklearn.feature_extraction.text import TfidfVectorizer #
from nltk import pos_tag 
from sklearn.metrics import pairwise_distances 
from nltk import word_tokenize 
from nltk.corpus import stopwords 

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package wordnet to /root/nltk_data...
[nltk_data]   Unzipping corpora/wordnet.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.


In [ ]:
from google.colab import files
data=files.upload() #загрузка датасета в google colab

Saving dialog_talk_agent.xlsx to dialog_talk_agent.xlsx


In [ ]:
data=pd.read_excel('dialog_talk_agent.xlsx')
data.ffill(axis = 0,inplace=True) # заполняем пропуски.
data.head(10)# просмотр данных

,Context,Text Response
0,Tell me about your personality,Just think of me as the ace up your sleeve.
1,I want to know you better,I can help you work smarter instead of harder
2,Define yourself,I can help you work smarter instead of harder
3,Describe yourself,I can help you work smarter instead of harder
4,tell me about yourself,I can help you work smarter instead of harder
5,all about you,I can help you work smarter instead of harder
6,tell me some stuff about you,I can help you work smarter instead of harder
7,talk some stuff about you,I can help you work smarter instead of harder
8,talk about yourself,I can help you work smarter instead of harder
9,about yourself,I can help you work smarter instead of harder


In [ ]:
# Нормализация текста

def preproc(string1):
    string1 = str(string1).lower() # приведение символов к нижнему регистру
    string_low = re.sub(r'[^ a-z]','',string1) # удаление спец. знаков
    tokenizer = nltk.word_tokenize(string_low) # токинезация текста
    lem=wordnet.WordNetLemmatizer() # лемматизация (процесс преобразования слова в его базовую форму)
    list_of_tags = pos_tag(tokenizer,tagset=None) # определение части речи, для выбора корня слова и его преобразования
    lem_word = []   # пустой список
    for token,token_pos in list_of_tags:
        if token_pos.startswith('V'):  # глагол
            Vpos = 'v'
        elif token_pos.startswith('J'): # прилагательное
            Vpos = 'a'
        elif token_pos.startswith('R'): # наречие
            Vpos ='r'
        else:
            Vpos = 'n' # существительное
        lem_token = lem.lemmatize(token,Vpos) # выполнение лемматизации
        lem_word.append(lem_token) # добавление лематации к таблице
    
    return " ".join(lem_word) # возвращает лемматизированные токены как предложение

In [ ]:
data['Lemmatized'] = data['Context'].apply(preproc) # Добавление колонки с преобразованными предложениями
stop = stopwords.words('english')# стопслова не несут смысловую нагрузку, однако их часто используют, желательно их исключить из словаря
CountVect = CountVectorizer() # Преобразование коллекции текстовых документов в матрицу подсчета токенов
var1 = CountVect.fit_transform(data['Lemmatized']).toarray()

In [ ]:
# все уникальные слова в датасете
diction = CountVect.get_feature_names()
data_bow = pd.DataFrame(var1, columns = diction)
data_bow.head()

,abort,about,absolutely,abysmal,actually,adore,advice,advise,affirmative,afraid,afternoon,again,age,agree,ah,ahah,ahaha,ahahah,ahahaha,ahead,all,almost,alone,already,alright,alrighty,also,always,amaze,amazing,an,and,angry,annoy,annoying,annul,answer,any,anymore,anything,...,whats,whazzup,when,where,wheres,which,who,whole,why,wife,will,wise,with,woah,wonderful,wooow,work,world,worry,worthless,would,wow,wrong,xd,ya,yap,ye,yea,yeah,year,yeh,yep,yes,yet,you,your,youre,yours,yourself,yup
0,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
3,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0
4,0,1,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,0,1,0


In [ ]:
def chat(string2):
    lem = preproc(string2)
    bow=CountVect.transform([lem]).toarray()
    cosine_value = 1- pairwise_distances(data_bow,bow, metric = 'cosine' )
    data['Similarity']=cosine_value
    val_index=cosine_value.argmax()
    print('ответ на вопрос', data['Text Response'].loc[val_index])
    print('вопрос из датасета', data[ 'Context'].loc[val_index])
    print('Совпадение с вопросом в датасете',data['Similarity'].loc[val_index]*100,"%")


In [ ]:
# мини-чат для теста
string_input = ''
print("напишите'stop' чтобы выйти")
while string_input!='stop':
    string_input = input()
    chat(string_input)
    

напишите'stop' чтобы выйти
hi
ответ на вопрос Hey!
вопрос из датасета hi
Совпадение с вопросом в датасете 100.0 %
how are you
ответ на вопрос Lovely, thanks.
вопрос из датасета how are you
Совпадение с вопросом в датасете 100.0 %
ahahha
ответ на вопрос Just think of me as the ace up your sleeve.
вопрос из датасета Tell me about your personality
Совпадение с вопросом в датасете 0.0 %
lol
ответ на вопрос You have a great laugh.
вопрос из датасета lol
Совпадение с вопросом в датасете 100.0 %
stop
ответ на вопрос Cancelled. Go on with the commands!
вопрос из датасета stop
Совпадение с вопросом в датасете 100.0 %
